### classify stackoverflow posts by considering each document a simple mean of its individual word embeddings, then using individual tag probabilities to assign tags to samples

In [1]:
import csv
import os
import re
import sys
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

Using TensorFlow backend.


In [2]:
SEED=np.random.randint(1,1000)
SEED

124

In [3]:
np.random.seed(SEED)

In [4]:
texts, labels = files_helper.read_stackoverflow_sample_small()

In [36]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10
TOKENIZER_FILTERS='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
TAG_PROB_THRESHOLD=0.05

In [6]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [7]:
[inverse_word_index[idx] for idx in sequences[0]]

['apply',
 'onclick',
 'event',
 'to',
 'an',
 'option',
 'i',
 'am',
 'using',
 'zend',
 'form',
 'to',
 'create',
 'a',
 'form',
 'i',
 'am',
 'also',
 'using',
 'for',
 'javascript',
 'this',
 'gt',
 'radio',
 'alone',
 'array',
 'label',
 'gt',
 'are',
 'you',
 'going',
 'to',
 'be',
 'taking',
 'part',
 'with',
 'anyone',
 'else',
 'required',
 'gt',
 'true',
 'onclick',
 'gt',
 'gt',
 'array',
 'yes',
 'gt',
 'yes',
 'no',
 'gt',
 'no',
 'at',
 'the',
 'moment',
 'the',
 'onclick',
 'event',
 'works',
 'if',
 'any',
 'option',
 'is',
 'selected',
 'how',
 'do',
 'i',
 'get',
 'it',
 'to',
 'work',
 'for',
 'just',
 'yes',
 'being',
 'selected']

In [8]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [9]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [10]:
# tag position => fraction of docs having that tag
tag_probabilities_index = tags_helper.get_probabilities_index(binary_labels)

In [11]:
# word => embedding
embeddings_index = files_helper.read_glove(d=EMBEDDING_DIM)

In [12]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [13]:
embedding_matrix.shape

(178778, 100)

In [58]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

print(type(sequences))
print(type(data))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

data.shape,labels_1.shape

<class 'list'>
<class 'numpy.ndarray'>


((32000, 1000), (32000, 1209))

In [46]:
seq_lengths_train = [len(seq) for seq in data]
seq_lengths_train[:10]

[213, 191, 765, 108, 149, 63, 480, 169, 185, 178]

In [49]:
%%time

num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           trainable = False))

model.add(GlobalAvgPool1D())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam')
# multi label needs a specific accuracy function


# loss doesn't get better after 5 epochs
model.fit(X_train, Y_train,
                    batch_size=1,
                    epochs=3,
                    verbose=1,
                    validation_data=(X_val,Y_val))

Train on 25600 samples, validate on 6400 samples
Epoch 1/3


ValueError: setting an array element with a sequence.

In [27]:
def get_scores(predicted_label_probabilities,
                        predicted_label_indices):
    
    active_indices = predicted_label_indices.ravel().nonzero()[0]
        
    predicted_label_scores = predicted_tag_probabilities[active_indices]
    
    predicted_label_scores_as_matrix = predicted_label_scores.reshape(1,-1)
    
    return predicted_label_scores_as_matrix
    

In [37]:
# sample result for a couple of test cases
num_test_cases = 10

for i in np.random.randint(low=0, high=len(Y_val), size=num_test_cases):

    actual_label_indices = Y_val[i].reshape(1,-1)
    actual_labels = lb.inverse_transform(actual_label_indices)
    actual_labels_tpl = actual_labels[0]
    
    predicted_tag_probabilities = model.predict(X_val[i].reshape(1,-1)).ravel()
              
    predicted_label_indices = tags_helper.get_predicted_indices_by_threshold(
        predicted_tag_probabilities,
        TAG_PROB_THRESHOLD)
       
#     predicted_label_indices = get_predicted_indices_by_tag_doc_fraction(
#         tag_probabilities_index,
#         predicted_tag_probabilities)
        
#     print(predicted_label_indices.nonzero()[0])
    
    predicted_label_scores_mat = get_scores(predicted_tag_probabilities,
                                       predicted_label_indices)
    
    predicted_label_scores_arr = predicted_label_scores_mat.ravel()
    
    predicted_labels = lb.inverse_transform(predicted_label_indices)

    predicted_labels_tpl = predicted_labels[0]
    
#     print(predicted_label_scores.ravel())
    
    tags_and_scores = sorted(list(zip(predicted_labels_tpl,predicted_label_scores_arr)), key=lambda tpl: tpl[1], reverse=True)
  

    print(actual_labels_tpl)
    print(tags_and_scores)
    print('\n')


('c#', 'powershell')
[('javascript', 0.091401227), ('java', 0.08990597), ('php', 0.084820308), ('c#', 0.082430623), ('android', 0.064892255), ('python', 0.063191071), ('jquery', 0.056961823)]


('connection', 'entity-framework', 'html', 'sql')
[('javascript', 0.090719461), ('c#', 0.089050427), ('java', 0.084624268), ('php', 0.083457969), ('python', 0.065069929), ('android', 0.063566953), ('ios', 0.061236028), ('jquery', 0.059151817)]


('ios', 'ipad', 'objective-c')
[('javascript', 0.090820253), ('java', 0.088617824), ('php', 0.084092028), ('c#', 0.083643287), ('android', 0.064506292), ('python', 0.063068613), ('jquery', 0.057059456)]


('c', 'macros')
[('java', 0.097104989), ('javascript', 0.096425377), ('php', 0.090629227), ('c#', 0.083535165), ('android', 0.069601841), ('python', 0.065857857), ('jquery', 0.059849642)]


('cordova',)
[('java', 0.10516103), ('javascript', 0.097404316), ('php', 0.092496224), ('c#', 0.074435994), ('android', 0.070711777), ('python', 0.063167028), ('jque

In [38]:
X_test_list = []

for validation_features in X_val:
    X_test_list.append(validation_features.reshape(1,-1))

X_test = np.vstack(X_test_list)    

In [39]:
Y_pred_lst = [model.predict(x_test.reshape(1,-1)) for x_test in X_test]

Y_pred_calculated_lst = [tags_helper.get_predicted_indices_by_threshold(y_pred,
                                                                        TAG_PROB_THRESHOLD) for y_pred in np.vstack(Y_pred_lst)]

Y_pred = np.vstack(Y_pred_calculated_lst)

Y_val.shape,Y_pred.shape

micro_f1 = metrics_helper.calculate_multilabel_metrics(Y_val,Y_pred)
print('micro-averaged F1 score (validation set) is {0}'.format(micro_f1))

micro_prec = metrics_helper.calculate_multilabel_metrics(Y_val, Y_pred, metric='precision')
print('micro-averaged precision score (validation set) is {0}'.format(micro_prec))

micro_rec = metrics_helper.calculate_multilabel_metrics(Y_val, Y_pred, metric='recall')
print('micro-averaged recall score (validation set) is {0}'.format(micro_rec))

micro-averaged F1 score (validation set) is 0.11470148375930499
micro-averaged precision score (validation set) is 0.07643896976483762
micro-averaged recall score (validation set) is 0.2296615301796649
